In [85]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.callbacks import EarlyStopping
import tensorflow_hub as hub

In [86]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)
print(tf.__version__)

Num GPUs Available:  1
2.10.0


In [87]:
keras_cache_dir = os.path.join(os.getcwd(),'keras_cache')
os.makedirs(keras_cache_dir, exist_ok = True)

tfhub_cache_dir = os.path.join(os.getcwd(),'tfhub_cache')
os.makedirs(tfhub_cache_dir, exist_ok = True)
os.environ['TFHUB_CACHE_DIR'] = tfhub_cache_dir

In [88]:
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
# if tf.test.gpu_device_name():
#     print('GPU found')
# else:
#     print('No GPU found')

In [89]:
classifier_model ="https://www.kaggle.com/models/google/inception-v3/frameworks/TensorFlow2/variations/classification/versions/2" #@param {type:"string"}

In [90]:
IMAGE_SHAPE = (299, 299)
BATCH = 32

In [91]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [92]:
data_dir = './data/archive/Train/'
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=IMAGE_SHAPE,
  batch_size=BATCH)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=IMAGE_SHAPE,
  batch_size=BATCH)

Found 39209 files belonging to 43 classes.
Using 31368 files for training.
Found 39209 files belonging to 43 classes.
Using 7841 files for validation.


In [93]:
# train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, IMAGE_SHAPE), y))
# val_ds = val_ds.map(lambda x, y: (tf.image.resize(x, IMAGE_SHAPE), y))

In [94]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

In [95]:
rescale = tf.keras.Sequential([
  layers.Rescaling(1./255)
])

In [96]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [97]:
classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_model, input_shape=IMAGE_SHAPE+(3,), trainable=True, arguments=dict(batch_norm_momentum=0.997))
])

classifier.trainable = False

inputs = keras.Input(shape=(IMAGE_SHAPE + (3,)))
x = data_augmentation(inputs)
x = rescale(x)

x = classifier(x)
outputs = layers.Dense(43, activation="softmax")(x)

model = keras.Model(inputs, outputs)

assert classifier.trainable == False  # All layers in `model` are now frozen
assert classifier.layers[0].trainable == False  # `trainable` is propagated recursively

model.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 299, 299, 3)]     0         
                                                                 
 sequential_16 (Sequential)  (None, 299, 299, 3)       0         
                                                                 
 sequential_17 (Sequential)  (None, 299, 299, 3)       0         
                                                                 
 sequential_18 (Sequential)  (None, 1001)              23853833  
                                                                 
 dense_9 (Dense)             (None, 43)                43086     
                                                                 
Total params: 23,896,919
Trainable params: 43,086
Non-trainable params: 23,853,833
_________________________________________________________________


In [98]:
tensorboard_logdir = os.path.join(os.getcwd(),'logs','tensorboard') # Let's log Tensorboard data to this folder
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorboard_logdir) # Create a Tensorboard "callback"
early_stopping = EarlyStopping()

In [99]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
  metrics=['accuracy'])

In [100]:
history = model.fit(train_ds, validation_data=val_ds, callbacks=[tensorboard_callback], epochs=25, batch_size=BATCH)

Epoch 1/25
981/981 [==============================] - 204s 203ms/step - loss: 3.0965 - accuracy: 0.5539 - val_loss: 2.9138 - val_accuracy: 0.6091
Epoch 2/25
981/981 [==============================] - 209s 213ms/step - loss: 2.7930 - accuracy: 0.6468 - val_loss: 3.0141 - val_accuracy: 0.6370
Epoch 3/25
981/981 [==============================] - 195s 198ms/step - loss: 2.7045 - accuracy: 0.6786 - val_loss: 2.9545 - val_accuracy: 0.6676
Epoch 4/25
981/981 [==============================] - 188s 192ms/step - loss: 2.6910 - accuracy: 0.6954 - val_loss: 3.2086 - val_accuracy: 0.6572
Epoch 5/25
981/981 [==============================] - 186s 190ms/step - loss: 2.6797 - accuracy: 0.7067 - val_loss: 3.2640 - val_accuracy: 0.6735
Epoch 6/25
981/981 [==============================] - 181s 185ms/step - loss: 2.6435 - accuracy: 0.7177 - val_loss: 3.5583 - val_accuracy: 0.6557
Epoch 7/25
981/981 [==============================] - 183s 186ms/step - loss: 2.6275 - accuracy: 0.7253 - val_loss: 3.3921 -

In [101]:
classifier.trainable = True
assert classifier.trainable == True
assert classifier.layers[0].trainable == True
model.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 299, 299, 3)]     0         
                                                                 
 sequential_16 (Sequential)  (None, 299, 299, 3)       0         
                                                                 
 sequential_17 (Sequential)  (None, 299, 299, 3)       0         
                                                                 
 sequential_18 (Sequential)  (None, 1001)              23853833  
                                                                 
 dense_9 (Dense)             (None, 43)                43086     
                                                                 
Total params: 23,896,919
Trainable params: 23,862,487
Non-trainable params: 34,432
_________________________________________________________________


In [102]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),  # Low learning rate
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
)

In [ ]:
history = model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10
981/981 [==============================] - 389s 396ms/step - loss: 0.3979 - accuracy: 0.9642 - val_loss: 0.3299 - val_accuracy: 0.9834
Epoch 2/10
981/981 [==============================] - 398s 406ms/step - loss: 0.3644 - accuracy: 0.9732 - val_loss: 0.3239 - val_accuracy: 0.9869
Epoch 3/10
981/981 [==============================] - 421s 430ms/step - loss: 0.3404 - accuracy: 0.9809 - val_loss: 0.3275 - val_accuracy: 0.9864
Epoch 4/10
981/981 [==============================] - 432s 440ms/step - loss: 0.3242 - accuracy: 0.9848 - val_loss: 0.3069 - val_accuracy: 0.9920
Epoch 5/10
981/981 [==============================] - 397s 405ms/step - loss: 0.3074 - accuracy: 0.9906 - val_loss: 0.2948 - val_accuracy: 0.9936
Epoch 7/10
981/981 [==============================] - 395s 403ms/step - loss: 0.3019 - accuracy: 0.9922 - val_loss: 0.2896 - val_accuracy: 0.9948
Epoch 8/10
981/981 [==============================] - 389s 396ms/step - loss: 0.2961 - accuracy: 0.9940 - val_loss: 0.2944 -

In [105]:
export_path = os.path.join(os.getcwd(), 'saved_models', 'german_sign_classifier_model')
model.save(export_path)